In [1]:
# Librerías
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from astroquery.mpc import MPC

In [2]:
# Conexión del API con el programa
url = 'https://cobs.si/api/obs_list.api?des=C/2023 A3&format=json&from_date=&to_date=&exclude_faint=False&exclude_not_accurate=False'
response = requests.get(url) 

if response.status_code == 200:
    content = response.json()
    print('✅ Base de datos actualizada.')
else:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Base de datos actualizada.


In [166]:
# Creación de Dataset
cometa_df = pd.DataFrame(content['objects'])
cometa_df.sort_values('obs_date', ascending=False).info()

<class 'pandas.core.frame.DataFrame'>
Index: 1151 entries, 0 to 1150
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       1151 non-null   object 
 1   obs_date                   1151 non-null   object 
 2   comet                      1151 non-null   object 
 3   observer                   1151 non-null   object 
 4   location                   528 non-null    object 
 5   extinction                 68 non-null     object 
 6   obs_method                 1151 non-null   object 
 7   comet_visibility           3 non-null      object 
 8   magnitude                  1151 non-null   object 
 9   conditions                 18 non-null     object 
 10  ref_catalog                1151 non-null   object 
 11  instrument_aperture        1151 non-null   object 
 12  instrument_type            1151 non-null   object 
 13  instrument_focal_ratio     1008 non-null   float64
 1

In [167]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 1151
Variables: 47


In [168]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
563,C,2024-05-04 21:48:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Mieczyslaw Leszek', 'last_name...","Dabrowa, Poland",None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,10.2,None,...,0.01,None,6.8,6.8,s,temperature: +10 C,None,NaN,,2024-05-09 14:02:40
669,V,2024-04-29 00:21:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Mike', 'last_name': 'Collins',...",Everton,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,9.4,None,...,None,None,None,None,None,,19.6,2.0,"TYC 4959-0618, TYC 4959-0727",2024-04-29 10:56:21
166,V,2024-06-20 15:21:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Virgilio', 'last_name': 'Gonan...","Udine, Italy",None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,10.3,None,...,None,None,None,None,None,,None,NaN,,2024-06-22 13:23:17
389,C,2024-05-16 13:12:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Andrew', 'last_name': 'Pearce'...",None,None,"{'key': 'V', 'name': 'Johnson/Bessel/Kron/Cous...",None,11.1,None,...,0.04,None,4.8,4.8,s,,None,NaN,,2024-05-17 13:50:45
1001,C,2023-09-06 19:40:48,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Pieter-Jan', 'last_name': 'Dek...",None,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,15.3,None,...,0.14,None,1.1,1.1,s,,None,NaN,,2023-09-07 20:24:30


In [169]:
# Métodos de observación
cometa_df.obs_method.apply(lambda registro: f'{registro['key']}: {registro['name']}').value_counts()

obs_method
C: Unfiltered total CCD magnitude            341
Z: CCD Visual equivalent                     292
S: In-Out method                             193
M: Modified-Out method                       174
V: Johnson/Bessel/Kron/Cousins V with CCD     76
B: Simple Out-Out method                      19
k: Kron/Cousins R with CCD                    18
H: Kron/Cousins I with CCD                    17
D: Johnson/Bessel/Kron/Cousins B with CCD     10
c: Unfiltered nuclear CCD magnitude            8
d: Astrodon G filter                           2
E: Extrafocal-Extinction method                1
Name: count, dtype: int64

In [170]:
# Con valores nulos
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas + 1}')

Registros: 1151
Variables: 48


In [171]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'])
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [172]:
# Info de nuevo data frame
curva_de_luz_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']]
curva_de_luz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151 entries, 0 to 1150
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1151 non-null   object        
 1   obs_date        1151 non-null   datetime64[ns]
 2   magnitude       1151 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 27.1+ KB


In [173]:
# Data Frame de la curva de luz
curva_de_luz_df.sample(5)

,obs_method_key,obs_date,magnitude
85,V,2024-07-04,10.9
933,Z,2024-02-16,13.9
182,S,2024-06-14,10.8
1098,Z,2023-04-30,17.1
7,M,2024-08-06,8.5


In [174]:
# Gráfica del los datos
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title='Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [175]:
# Variables a graficar
tiempo = curva_de_luz_df.obs_date
obs_method = curva_de_luz_df.obs_method_key.unique()

# Inicializar la figura con subplots
fig = make_subplots(rows=4, cols=3, subplot_titles= obs_method)

# Agregar trazas a los subplots
for metodo, fila, columna  in zip(obs_method, [1]*3 + [2]*3 + [3]*3, [1,2,3]*3):
    fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == metodo], mode="markers",name = metodo), row = columna, col = fila)

# Actualizar el diseño de los subplots
fig.update_layout(title_text = 'Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)', template = 'plotly_dark')
fig.update_yaxes(autorange="reversed")

# Mostrar la figura
fig.show()

In [176]:
# Ephemeris
eph = MPC.get_ephemeris('C/2023 A3')
ephemeris_df = eph.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower()
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper motion,direction
0,2024-08-24 16:47:50,161.620417,-2.260000,1.858,0.922,15.3,16.8,9.1,25.59,217.0
1,2024-08-25 16:47:50,161.517083,-2.397222,1.844,0.903,14.7,16.4,9.0,25.92,217.2
2,2024-08-26 16:47:50,161.411250,-2.535556,1.830,0.884,14.0,16.1,8.9,26.26,217.5
3,2024-08-27 16:47:50,161.303750,-2.675000,1.815,0.865,13.4,15.7,8.8,26.61,217.8
4,2024-08-28 16:47:50,161.193750,-2.815556,1.799,0.845,12.9,15.5,8.7,26.96,218.2
5,2024-08-29 16:47:50,161.081250,-2.957500,1.783,0.826,12.4,15.2,8.6,27.31,218.5
6,2024-08-30 16:47:50,160.966250,-3.100556,1.766,0.806,12.0,15.1,8.5,27.66,219.0
7,2024-08-31 16:47:50,160.848750,-3.244167,1.748,0.787,11.7,15.1,8.4,28.01,219.4
8,2024-09-01 16:47:50,160.728750,-3.388889,1.730,0.767,11.4,15.1,8.3,28.35,219.9
9,2024-09-02 16:47:50,160.606250,-3.534444,1.711,0.748,11.3,15.3,8.2,28.67,220.3


In [177]:
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           21 non-null     datetime64[ns]
 1   ra             21 non-null     float64       
 2   dec            21 non-null     float64       
 3   delta          21 non-null     float64       
 4   r              21 non-null     float64       
 5   elongation     21 non-null     float64       
 6   phase          21 non-null     float64       
 7   v              21 non-null     float64       
 8   proper motion  21 non-null     float64       
 9   direction      21 non-null     float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 1.8 KB


In [178]:
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           21 non-null     datetime64[ns]
 1   ra             21 non-null     float64       
 2   dec            21 non-null     float64       
 3   delta          21 non-null     float64       
 4   r              21 non-null     float64       
 5   elongation     21 non-null     float64       
 6   phase          21 non-null     float64       
 7   v              21 non-null     float64       
 8   proper motion  21 non-null     float64       
 9   direction      21 non-null     float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 1.8 KB


In [179]:
curva_de_luz_ephemeris_df = ephemeris_df[['date', 'delta','r', 'phase']]
curva_de_luz_ephemeris_df = curva_de_luz_ephemeris_df.rename(columns = {'date':'obs_date'})
curva_de_luz_ephemeris_df

,obs_date,delta,r,phase
0,2024-08-24,1.858,0.922,16.8
1,2024-08-25,1.844,0.903,16.4
2,2024-08-26,1.830,0.884,16.1
3,2024-08-27,1.815,0.865,15.7
4,2024-08-28,1.799,0.845,15.5
5,2024-08-29,1.783,0.826,15.2
6,2024-08-30,1.766,0.806,15.1
7,2024-08-31,1.748,0.787,15.1
8,2024-09-01,1.730,0.767,15.1
9,2024-09-02,1.711,0.748,15.3


In [194]:
curva_de_luz_df[curva_de_luz_df.obs_date.dt.year == 2024]

,obs_method_key,obs_date,magnitude
0,Z,2024-08-12,8.2
1,M,2024-08-11,8.9
2,Z,2024-08-11,8.4
3,S,2024-08-10,9.0
4,Z,2024-08-10,8.4
...,...,...,...
980,C,2024-01-07,13.7
981,M,2024-01-07,13.7
982,S,2024-01-07,12.7
983,C,2024-01-05,13.8


In [182]:
curva_de_luz_df.merge(curva_de_luz_ephemeris_df, on='obs_date')

,obs_method_key,obs_date,magnitude,delta,r,phase
